# Batch computation

<em style="color:gray;">Copyright (c) 2024 QuAIR team. All Rights Reserved.</em>

This tutorial involves how to use batch computation to simplify a series of inputs in QuAIRKit.

**Table of Contents**
- [Batch computation in quantum circuit](#Batch-computation-in-quantum-circuit)
- [Batch computation in measurement](#Batch-computation-in-measurement)

In [1]:
import torch

import quairkit as qkit
from quairkit import Circuit
from quairkit.database import *
from quairkit.loss import ExpecVal, Measure

qkit.set_dtype("complex128")

## Batch computation in quantum circuit

`Circuit` class in QuAIRKit supports add batched parameters and gates to the circuit. 

For parameterized gates like $R_x(\theta)$, $R_y(\theta)$, $R_z(\theta)$, one can add batched parameters to the circuit by passing a 3-dimensional tensor to gate function, where the 3 dimensions are:

- *len(qubits_idx)* : the number of qubits acted by the gates;
- *batch_size* : the number of batched parameters;
- *num_acted_param* : the number of parameters that characterize the gate. For example, *num_acted_param* for Ry gate is 1 and that for universal three qubit gate is 15.

Here is an example of batched parameters as an input onto a parameterized quantum circuit.

In [2]:
num_qubits = 2
batch_size = 3

list_x = torch.rand(num_qubits * batch_size * 1)  # num_acted_param=1
cir = Circuit(num_qubits)
cir.rx(param=list_x)  # set Rx gate
print(f"Quantum circuit output: {cir()}")

# this is equivalent to below code
# list_x = list_x.view(num_qubits, batch_size, 1).permute([1, 0, 2])
# for param in list_x:
#     cir_1 = Circuit(num_qubits)
#     cir_1.rx(param=param)
#     print(f"Quantum circuit output for adding one Rx gate: {cir_1()}")

Quantum circuit output: 
-----------------------------------------------------
 Backend: default-pure
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [3]

 # 0:
[ 0.96+0.j    0.  -0.23j  0.  -0.19j -0.04+0.j  ]
 # 1:
[ 0.88+0.j    0.  -0.41j  0.  -0.22j -0.11+0.j  ]
 # 2:
[ 0.99+0.j    0.  -0.17j  0.  -0.03j -0.  +0.j  ]
-----------------------------------------------------



For oracles stored as torch.Tensor, one can add batched matrices to the circuit by `oracle`.

In [3]:
cir_ora = Circuit(2)
list_unitary = random_unitary(1, size=batch_size)
print(f"The shape of oracle unitary: {list_unitary.shape}")

cir_ora.oracle(list_unitary, [1])
print(f"Quantum circuit output: {cir_ora()}")

# this is equivalent to below code
# for idx, unitary in enumerate(list_unitary):
#     cir_ora2 = Circuit(2)
#     cir_ora2.oracle(unitary, [1])
#     print(f"Quantum circuit {idx}: {cir_ora2()}")

The shape of oracle unitary: torch.Size([3, 2, 2])
Quantum circuit output: 
-----------------------------------------------------
 Backend: default-pure
 System dimension: [2, 2]
 System sequence: [1, 0]
 Batch size: [3]

 # 0:
[ 0.58+0.55j  0.  +0.j   -0.27-0.54j  0.  +0.j  ]
 # 1:
[-0.29-0.82j  0.  +0.j   -0.34-0.37j  0.  +0.j  ]
 # 2:
[-0.77-0.02j  0.  +0.j    0.11+0.62j  0.  +0.j  ]
-----------------------------------------------------



QuAIRKit also supports batched channels through batching their Kraus or Choi operators. One can add batched channels to the circuit via `kraus_channel` or `choi_channel`.  Notice that Kraus representation is recommended in batch computation.

In [4]:
cir_kra = Circuit(2)
list_kraus = random_channel(num_qubits=1, size=batch_size)
cir_kra.kraus_channel(list_kraus, [0])
print(f"Kraus channel: {cir_kra()}")
output_state = cir_kra()

# this is equivalent to below code
# for idx, kraus in enumerate(list_kraus):
#     cir_kra2 = Circuit(2)
#     cir_kra2.kraus_channel(kraus, [0])
#     print(f"Kraus channel {idx}: {cir_kra2()}")

Kraus channel: 
-----------------------------------------------------
 Backend: default-mixed
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [3]

 # 0:
[[0.58+0.j   0.  +0.j   0.01-0.05j 0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]
 [0.01+0.05j 0.  +0.j   0.42+0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]]
 # 1:
[[ 0.11+0.j    0.  +0.j   -0.11-0.12j  0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]
 [-0.11+0.12j  0.  +0.j    0.89+0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]]
 # 2:
[[ 0.35+0.j    0.  +0.j   -0.3 +0.33j  0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]
 [-0.3 -0.33j  0.  +0.j    0.65+0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]]
-----------------------------------------------------



In [5]:
cir_cho = Circuit(2)
list_choi = random_channel(num_qubits=1, target="choi", size=batch_size)
cir_cho.choi_channel(list_choi, [1])
print(f"Choi channel: {cir_cho()}")

# this is equivalent to below code
# for idx, choi in enumerate(list_choi):
#     cir_cho2 = Circuit(2)
#     cir_cho2.choi_channel(choi, [0])
#     print(f"Choi channel {idx}: {cir_cho2()}")

Choi channel: 
-----------------------------------------------------
 Backend: default-mixed
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [3]

 # 0:
[[0.26+0.j   0.26+0.27j 0.  +0.j   0.  +0.j  ]
 [0.26-0.27j 0.74-0.j   0.  +0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]]
 # 1:
[[0.14-0.j   0.12-0.18j 0.  +0.j   0.  +0.j  ]
 [0.12+0.18j 0.86+0.j   0.  +0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]]
 # 2:
[[ 0.65+0.j   -0.29+0.26j  0.  +0.j    0.  +0.j  ]
 [-0.29-0.26j  0.35+0.j    0.  +0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]]
-----------------------------------------------------



Mathematical property of Kraus operators is checked.

One can then check that this circuit preserves the trace.

In [6]:
tr = output_state.trace()
torch.allclose(tr, torch.ones_like(tr))

True

For clarity, the following figure illustrates how batch computation works in quantum circuits.

<figure style="text-align: center;">
  <img src="figures/batch_demo.jpg" alt="alt text" width="700"/>
  <figcaption>Fig.1: Depiction of batched quantum circuits on single input state.</figcaption>
</figure>


The code of these circuits is given as follows 

In [7]:
rho = random_state(1)
list_x = torch.rand(batch_size)
list_depo = torch.stack(
    [depolarizing_kraus(torch.rand(1)) for _ in list(range(batch_size))]
)

batch_cir = Circuit(1)
batch_cir.ry()
batch_cir.rz(param=list_x)
batch_cir.kraus_channel(list_depo, 0)
batch_cir.ry()
batch_cir.rz(param=list_x)
print(f"Output state: {batch_cir(rho)}")

Output state: 
-----------------------------------------------------
 Backend: default-mixed
 System dimension: [2]
 System sequence: [0]
 Batch size: [3]

 # 0:
[[ 0.42+0.j   -0.05+0.06j]
 [-0.05-0.06j  0.58+0.j  ]]
 # 1:
[[ 0.46-0.j   -0.03+0.03j]
 [-0.03-0.03j  0.54-0.j  ]]
 # 2:
[[ 0.34-0.j   -0.11+0.12j]
 [-0.11-0.12j  0.66-0.j  ]]
-----------------------------------------------------



## Batch computation in measurement

Measurement in QuAIRKit also support batch computation. We start with an observable represented by `Hamiltonian` and a projection valued measure (PVM).

In [8]:
H = random_hamiltonian_generator(num_qubits)
print(f"Hamiltonian: {H.pauli_str}")

Hamiltonian: [[0.8891666269428207, 'X0'], [0.7368808573711985, 'Y0']]


One can call the `expec_val` of `State` class, or implement the neural network module `ExpecVal` on batched states.

In [9]:
print(f"Output state: {output_state}")
op = ExpecVal(H)
print(f"expectation value: {op(output_state)}")
# this is equivalent to below code
# for state in output_state:
#     print(f"expectation value of each: {op(state)}")

print(f"expectation value: {output_state.expec_val(H)}")
# return the expectation value of each Pauli term
print(
    f"expectation value of each Pauli term:\n{output_state.expec_val(H, decompose=True)}"
)

Output state: 
-----------------------------------------------------
 Backend: default-mixed
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [3]

 # 0:
[[0.58+0.j   0.  +0.j   0.01-0.05j 0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]
 [0.01+0.05j 0.  +0.j   0.42+0.j   0.  +0.j  ]
 [0.  +0.j   0.  +0.j   0.  +0.j   0.  +0.j  ]]
 # 1:
[[ 0.11+0.j    0.  +0.j   -0.11-0.12j  0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]
 [-0.11+0.12j  0.  +0.j    0.89+0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]]
 # 2:
[[ 0.35+0.j    0.  +0.j   -0.3 +0.33j  0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]
 [-0.3 -0.33j  0.  +0.j    0.65+0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]]
-----------------------------------------------------

expectation value: tensor([ 0.0779, -0.0278, -1.0180])
expectation value: tensor([ 0.0779, -0.0278, -1.0180])
expectation value of each Pauli term:
tensor([[ 0.0094, 

Similarly, to measure the output state, one can call the `measure` of `State` class, or implement the neural network module `Measure` on batched states. The following code measures the second qubit of the output state.

In [10]:
output_state = cir_kra()
basis = random_unitary(1).unsqueeze(-1)
pvm = basis @ basis.mH
print(f"The shape of PVM: {pvm.shape}")

op = Measure(pvm)
print(f"expectation value: {op(output_state, [0])}")
# this is equivalent to below code
# for state in output_state:
#     print(f"expectation value: {op(state, [0])}")

print(f"expectation value: {output_state.measure([0], measure_op=pvm)}")

The shape of PVM: torch.Size([2, 2, 2])
expectation value: tensor([[0.4500, 0.5500],
        [0.4740, 0.5260],
        [0.5218, 0.4782]])
expectation value: tensor([[0.4500, 0.5500],
        [0.4740, 0.5260],
        [0.5218, 0.4782]])


One can also keep the collapsed states after the measurement by setting `keep_state = True`.

---

*Table: A reference of notation conventions in this tutorial.*

| Symbol        | Variant          | Description                               |
|:---------------:|:------------------:|-------------------------------------------|
| $R_{x/y/z}(\theta)$     |       |rotation gates about the $X$/$Y$/$Z$-axis  |
|$\rho_{\text{in}}$||input quantum state|
|$\rho_{\text{out}}$|$\rho_{\text{out},1}$, $\rho_{\text{out},2}$, $\rho_{\text{out},3}$| output quantum state|
|$\mathcal{N}$|$\mathcal{N}_1$, $\mathcal{N}_2$, $\mathcal{N}_3$|quantum channel  |


In [11]:
qkit.print_info()


---------VERSION---------
quairkit: 0.4.2
torch: 2.8.0+cu128
torch cuda: 12.8
numpy: 2.2.6
scipy: 1.15.3
matplotlib: 3.10.6
---------SYSTEM---------
Python version: 3.10.18
OS: Linux
OS version: #1 SMP Tue Nov 5 00:21:55 UTC 2024
---------DEVICE---------
CPU:  13th Gen Intel(R) Core(TM) i9-13980HX
GPU: (0) NVIDIA GeForce RTX 4090 Laptop GPU
